In [81]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Neural_Network_Charity_Analysis/Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [82]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'. Also dropping 'STATUS' AND 'SPECIAL_CONSIDERATIONS' for potential optimization...
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'], 1)
application_df.head()

C:\Users\hrcrt\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [83]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [84]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [85]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [86]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [87]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [88]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\hrcrt\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [89]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [90]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop("IS_SUCCESSFUL", axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [91]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [116]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 30
nodes_hidden_layer2 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 30)                1230      
                                                                 
 dense_33 (Dense)            (None, 15)                465       
                                                                 
 dense_34 (Dense)            (None, 1)                 16        
                                                                 
Total params: 1,711
Trainable params: 1,711
Non-trainable params: 0
_________________________________________________________________


In [117]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [119]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_less_noise/",exist_ok=True)
checkpoint_path = "checkpoints_less_noise/weights.{epoch:02d}.hdf5"

In [120]:
# Create a callback that saves the model's weight every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 572us/step - loss: 0.5829 - accuracy: 0.7124
Epoch 2/100
804/804 [==============================] - 0s 577us/step - loss: 0.5556 - accuracy: 0.7287
Epoch 3/100
804/804 [==============================] - 0s 572us/step - loss: 0.5523 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 0s 571us/step - loss: 0.5506 - accuracy: 0.7293
Epoch 5/100
776/804 [===========================>..] - ETA: 0s - loss: 0.5491 - accuracy: 0.7315
Epoch 00005: saving model to checkpoints_less_noise\weights.05.hdf5
804/804 [==============================] - 0s 592us/step - loss: 0.5495 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 0s 582us/step - loss: 0.5483 - accuracy: 0.7313
Epoch 7/100
804/804 [==============================] - 0s 586us/step - loss: 0.5476 - accuracy: 0.7312
Epoch 8/100
804/804 [==============================] - 1s 631us/step - loss: 0.5468 - accuracy: 0.7323
Epoch 9/100
804/804 [==

In [121]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5567 - accuracy: 0.7311 - 139ms/epoch - 518us/step
Loss: 0.5566715598106384, Accuracy: 0.7310787439346313


In [97]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation_less_noise.h5")

Adjusting number of neurons to existing hidden layers...

In [122]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 60
nodes_hidden_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 60)                2460      
                                                                 
 dense_36 (Dense)            (None, 30)                1830      
                                                                 
 dense_37 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4,321
Trainable params: 4,321
Non-trainable params: 0
_________________________________________________________________


In [123]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [124]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints_add_neurons/",exist_ok=True)
checkpoint_path = "checkpoints_add_neurons/weights.{epoch:02d}.hdf5"

In [125]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 575us/step - loss: 0.5710 - accuracy: 0.7182
Epoch 2/100
804/804 [==============================] - 0s 574us/step - loss: 0.5553 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 0s 597us/step - loss: 0.5517 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 0s 604us/step - loss: 0.5506 - accuracy: 0.7310
Epoch 5/100
804/804 [==============================] - 0s 580us/step - loss: 0.5492 - accuracy: 0.7308
Epoch 6/100
804/804 [==============================] - 0s 574us/step - loss: 0.5484 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 0s 577us/step - loss: 0.5476 - accuracy: 0.7318
Epoch 8/100
804/804 [==============================] - 0s 589us/step - loss: 0.5471 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 0s 574us/step - loss: 0.5462 - accuracy: 0.7314
Epoch 10/100
804/804 [==============================] - 0s 584us/step - l

In [126]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5584 - accuracy: 0.7293 - 153ms/epoch - 571us/step
Loss: 0.5584197640419006, Accuracy: 0.7293294668197632


In [127]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation_add_neurons.h5")

Add additional hidden layer

In [128]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 60
nodes_hidden_layer2 = 30
nodes_hidden_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 60)                2460      
                                                                 
 dense_39 (Dense)            (None, 30)                1830      
                                                                 
 dense_40 (Dense)            (None, 20)                620       
                                                                 
 dense_41 (Dense)            (None, 1)                 21        
                                                                 
Total params: 4,931
Trainable params: 4,931
Non-trainable params: 0
_________________________________________________________________


In [129]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [130]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimization_third_layer/", exist_ok=True)
checkpoint_path = "checkpoints_optimization_third_layer/weights.{epoch:02d}hdf5"

In [131]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback], verbose=1)

Epoch 1/100
804/804 [==============================] - 1s 619us/step - loss: 0.5679 - accuracy: 0.7217
Epoch 2/100
804/804 [==============================] - 0s 614us/step - loss: 0.5528 - accuracy: 0.7286
Epoch 3/100
804/804 [==============================] - 1s 637us/step - loss: 0.5503 - accuracy: 0.7317
Epoch 4/100
804/804 [==============================] - 0s 620us/step - loss: 0.5480 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 1s 632us/step - loss: 0.5477 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 0s 620us/step - loss: 0.5465 - accuracy: 0.7340
Epoch 7/100
804/804 [==============================] - 1s 647us/step - loss: 0.5462 - accuracy: 0.7318
Epoch 8/100
804/804 [==============================] - 1s 654us/step - loss: 0.5454 - accuracy: 0.7342
Epoch 9/100
804/804 [==============================] - 1s 652us/step - loss: 0.5451 - accuracy: 0.7326
Epoch 10/100
804/804 [==============================] - 1s 643us/step - l

In [132]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5619 - accuracy: 0.7286 - 155ms/epoch - 578us/step
Loss: 0.5619198083877563, Accuracy: 0.7286297082901001


In [133]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation_third_layer.h5")

Changing activation functions for hidden layers

In [134]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 60
nodes_hidden_layer2 = 30
nodes_hidden_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="tanh", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 60)                2460      
                                                                 
 dense_43 (Dense)            (None, 30)                1830      
                                                                 
 dense_44 (Dense)            (None, 20)                620       
                                                                 
 dense_45 (Dense)            (None, 1)                 21        
                                                                 
Total params: 4,931
Trainable params: 4,931
Non-trainable params: 0
_________________________________________________________________


In [135]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [136]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimization_change_functions/", exist_ok=True)
checkpoint_path = "checkpoints_optimization_change_functions/weights.{epoch:02d}hdf5"

In [137]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback], verbose=1)

Epoch 1/100
804/804 [==============================] - 1s 631us/step - loss: 0.5680 - accuracy: 0.7233
Epoch 2/100
804/804 [==============================] - 1s 674us/step - loss: 0.5553 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 1s 628us/step - loss: 0.5520 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 1s 632us/step - loss: 0.5505 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 1s 639us/step - loss: 0.5484 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 1s 632us/step - loss: 0.5472 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 1s 654us/step - loss: 0.5465 - accuracy: 0.7337
Epoch 8/100
804/804 [==============================] - 1s 658us/step - loss: 0.5456 - accuracy: 0.7342
Epoch 9/100
804/804 [==============================] - 1s 641us/step - loss: 0.5450 - accuracy: 0.7316
Epoch 10/100
804/804 [==============================] - 1s 654us/step - l

In [138]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5602 - accuracy: 0.7278 - 154ms/epoch - 573us/step
Loss: 0.5602473616600037, Accuracy: 0.7278134226799011


In [139]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation_change_functions.h5")